# Helpdesk Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app12-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@7f1f77e9

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@434083ee

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val shift = hiveCxt.table("sqlzoo.Shift")
val staff = hiveCxt.table("sqlzoo.Staff")
val issue = hiveCxt.table("sqlzoo.Issue")
val shift_type = hiveCxt.table("sqlzoo.Shift_type")
val level = hiveCxt.table("sqlzoo.Level")
val customer = hiveCxt.table("sqlzoo.Customer")
val caller = hiveCxt.table("sqlzoo.Caller")

shift: DataFrame = [shift_date: string, shift_type: string ... 4 more fields]
staff: DataFrame = [staff_code: string, first_name: string ... 2 more fields]
issue: DataFrame = [call_date: string, call_ref: int ... 5 more fields]
shift_type: DataFrame = [shift_type: string, start_time: string ... 1 more field]
level: DataFrame = [level_code: int, manager: string ... 2 more fields]
customer: DataFrame = [company_ref: int, company_name: string ... 6 more fields]
caller: DataFrame = [caller_id: int, company_ref: int ... 2 more fields]

## 6.
List the Company name and the number of calls for those companies with more than 18 calls.

```
+------------------+----+
| Company_name     | cc |
+------------------+----+
| Gimmick Inc.     | 22 |
| Hamming Services | 19 |
| High and Co.     | 20 |
+------------------+----+
```

In [5]:
(issue.join(caller, "Caller_id")
 .join(customer, "Company_ref")
 .groupBy("Company_name")
 .agg(count("Call_ref").alias("cc"))
 .filter(col("cc")>18)
 .showHTML())

Company_name,cc
Hamming Services,19
Gimmick Inc.,22
High and Co.,20


## 7.
Find the callers who have never made a call. Show first name and last name

```
+------------+-----------+
| first_name | last_name |
+------------+-----------+
| David      | Jackson   |
| Ethan      | Phillips  |
+------------+-----------+
```

In [6]:
(caller.join(issue, "Caller_id", joinType="left_anti")
 .select("First_name", "Last_name")
 .showHTML())

First_name,Last_name
David,Jackson
Ethan,Phillips


## 8.
For each customer show: Company name, contact name, number of calls where the number of calls is fewer than 5

```
+--------------------+------------+-----------+----+
| Company_name       | first_name | last_name | nc |
+--------------------+------------+-----------+----+
| Pitiable Shipping  | Ethan      | McConnell |  4 |
| Rajab Group        | Emily      | Cooper    |  4 |
| Somebody Logistics | Ethan      | Phillips  |  2 |
+--------------------+------------+-----------+----+
```

In [7]:
(issue.join(caller.select("Caller_id", "Company_ref"), "Caller_id")
 .join(customer, "Company_ref")
 .groupBy("Company_ref", "Company_name", "Contact_id")
 .agg(count("Call_ref").alias("nc"))
 .filter(col("nc") < 5)
 .join(caller, (col("Contact_id")===col("Caller_id")))
 .select("Company_name", "First_name", "Last_name", "nc")
 .showHTML())

Company_name,First_name,Last_name,nc
Pitiable Shipping,Ethan,McConnell,4
Somebody Logistics,Ethan,Phillips,2
Rajab Group,Emily,Cooper,4


## 9.
For each shift show the number of staff assigned. Beware that some roles may be NULL and that the same person might have been assigned to multiple roles (The roles are 'Manager', 'Operator', 'Engineer1', 'Engineer2').

```
+------------+------------+----+
| Shift_date | Shift_type | cw |
+------------+------------+----+
| 2017-08-12 | Early      |  4 |
| 2017-08-12 | Late       |  4 |
| 2017-08-13 | Early      |  3 |
| 2017-08-13 | Late       |  2 |
| 2017-08-14 | Early      |  4 |
| 2017-08-14 | Late       |  4 |
| 2017-08-15 | Early      |  4 |
| 2017-08-15 | Late       |  4 |
| 2017-08-16 | Early      |  4 |
| 2017-08-16 | Late       |  4 |
+------------+------------+----+
```

In [8]:
(shift.selectExpr(
    "Shift_date", "Shift_type", 
    "stack(4, 'Manager', Manager, 'Operator', Operator, " + 
    "'Engineer1', Engineer1, 'Engineer2', Engineer2) as (role, p)")
 .dropDuplicates("Shift_date", "Shift_type", "p")
 .na.drop()
 .groupBy("Shift_date", "Shift_type")
 .agg(count("role").alias("cw"))
 .orderBy("Shift_date", "Shift_type")
 .showHTML())

Shift_date,Shift_type,cw
2017-08-12,Early,4
2017-08-12,Late,4
2017-08-13,Early,3
2017-08-13,Late,2
2017-08-14,Early,4
2017-08-14,Late,4
2017-08-15,Early,4
2017-08-15,Late,4
2017-08-16,Early,4
2017-08-16,Late,4


## 10.
Caller 'Harry' claims that the operator who took his most recent call was abusive and insulting. Find out who took the call (full name) and when.

```
+------------+-----------+---------------------+
| first_name | last_name | call_date           |
+------------+-----------+---------------------+
| Emily      | Best      | 2017-08-16 10:25:00 |
+------------+-----------+---------------------+
```

In [9]:
(caller.filter(col("First_name")==="Harry")
 .select("Caller_id")
 .join(issue, "Caller_id")
 .join(staff, (issue("Taken_by")===staff("Staff_code")))
 .orderBy(col("Call_date").desc)
 .select("First_name", "Last_name", "Call_date")
 .limit(1)
 .showHTML())

First_name,Last_name,Call_date
Emily,Best,2017-08-16 10:25:00.0


In [10]:
spark.stop()